# 章节3-1 使用Urllib
爬取CSDN（网址：https://edu.csdn.net/course/detail/29493 ）的一个课程页，并自动提取出QQ群。

In [ ]:
import urllib.request
import re

data = urllib.request.urlopen("https://edu.csdn.net/course/detail/29493").read().decode('utf8')
pat = '<span class="realname" data-v-a39d224e>(.*?)</span>'
result = re.compile(pat).findall(str(data))
print(result)

# 章节4-1 timeout设置
循环爬取首页（网址:https://edu.hellobi.com/ ），0.5秒无响应超时异常。

In [ ]:
import urllib.request

# urllib.request.urlretrieve("https://edu.hellobi.com/", filename="1.html")
# urllib.request.urlcleanup()
for i in range(10):
    try:
        file = urllib.request.urlopen("https://edu.hellobi.com/", timeout = 0.5)
        print(file.getcode())
    except Exception as e:
        print("访问{0}出现异常：{1}".format(file.geturl(), str(e)))
# file.geturl()

# 章节4-2 网址有中文
爬取百度（网址：http://www.baidu.com/ ）查找“Python”和“计算机”的查询结果。

In [ ]:
import urllib.request

keywd1 = "python"
keywd2 = urllib.request.quote("计算机")
url1 = "http://www.baidu.com/s?wd=" + keywd1
url2 = "http://www.baidu.com/s?wd=" + keywd2
req1 = urllib.request.Request(url1)
data1 = urllib.request.urlopen(req1).read()
req2= urllib.request.Request(url2)
data2 = urllib.request.urlopen(req2).read()
with open("1.html", "wb") as f:
    f.write(data1)
with open("2.html", "wb") as f:
    f.write(data2)

# 章节4-3 使用post
向网页（网址：http://www.iqianyue.com/mypost/ ）提交。

In [ ]:
import urllib.request
import urllib.parse

url = "http://www.iqianyue.com/mypost/"
mydata = urllib.parse.urlencode({"name":"test@test.com", "pass":"123456jkl"}).encode("utf-8")
req = urllib.request.Request(url, mydata)
print(data, req)
result = urllib.request.urlopen(req).read()
print(result.decode("utf-8"))
# with open("1.html", "wb") as fl:
#     fl.write(result)

# 章节4-4 异常处理

In [ ]:
import urllib.error
import urllib.request

try:
    result = urllib.request.urlopen("http://blog.csdn.net/ss12rwew")
except urllib.error.URLError as e:
    if hasattr(e, "code"):
        print(e.code)
    if hasattr(e, "reason"):
        print(e.reason)

# 章节4-5 使用urllib.request.urlretrieve
爬取新浪新闻（网址：https://news.sina.com.cn/ ）

In [ ]:
import urllib.request
import re

data = urllib.request.urlopen("https://news.sina.com.cn/").read()
data2 = data.decode("utf-8", "ignore")
pat = 'href="(https://news.sina.com.cn/.*?)"'
allurl = re.compile(pat).findall(data2)
# print(allurl)
for i in range(len(allurl)):
    try:
        print("第{}次爬取".format(i+1))
        file = "sinanews/" + str(i+1) + ".html"
        urllib.request.urlretrieve(allurl[i], file)
        print("----成功----")
    except urllib.error.HTTPError as e:
        print(str(e.code) + ": " + e.reason)
    except Exception as e:
        print("失败：{}".format(str(e)))

# 章节5-1 爬虫防屏蔽手段-代理服务器

In [ ]:
import urllib.request
import re

url = "http://blog.csdn.net/"
headers = ("user-agent", "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36")
opener = urllib.request.build_opener()
opener.addheaders = [headers]
urllib.request.install_opener(opener)
data = urllib.request.urlopen(url).read().decode("utf-8", "ignore")
print("output: " + data)

In [ ]:
import urllib.request

def use_proxy(url, proxy_addr):
    proxy = urllib.request.ProxyHandler({"http": proxy_addr})
    opener = urllib.request.build_opener(proxy, urllib.request.HTTPHandler)
    urllib.request.install_opener(opener)
    data = urllib.request.urlopen(url).read().decode("utf-8", "ignore")
    return data

proxy_addr = "47.107.160.99:8118"
url = "http://www.baidu.com"
data = use_proxy(url, proxy_addr)
print("output: " + data)

爬取快代理免费代理IP

In [ ]:
import requests
import time
import random
from lxml import etree
from fake_useragent import UserAgent
from queue import Queue
from threading import Thread


def get_ip():
    while True:
        if not q.empty():
        	# 验证IP是否可用网址
            url = 'http://httpbin.org/get'
            proxies = q.get()
            try:
                html = requests.get(url, headers=headers, proxies=proxies, timeout=5).text
                print('ip可以用')
                with open('ip.txt','a')as f:
                    f.write(str(proxies))
                    f.write('\n')
            except:
                print('ip不可用，下一个\t')
        else:
            break

def main():
    t_list = []
    for i in range(5):
        t = Thread(target=get_ip)
        t_list.append(t)
        t.start()

    for t in t_list:
        t.join()


ip_list = []
q = Queue()
for i in range(1, 10):
    url = 'https://www.kuaidaili.com/free/inha/{}'.format(i)
    print(url)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36',
    }
    # proxies = {'http': 'http://211.159.219.225:8118', 'https': 'https://211.159.219.225:8118'}
    # html = requests.get(url, headers=headers,proxies=proxies).text
    html = requests.get(url, headers=headers).text
    # print(html)
    parse_html = etree.HTML(html)
    tr_list = parse_html.xpath('//*[@id="list"]/table/tbody/tr')
    # 延迟访问6到11秒。
    sleep = random.randint(6, 11)
    print(f'等待{sleep}秒')
    time.sleep(sleep)
    print('开始')
    for tr in tr_list[1:]:
        ip = tr.xpath('./td[1]/text()')[0]
        port = tr.xpath('./td[2]/text()')[0]
        proxies = {
            'http': f'http://{ip}:{port}',
            'https': f'https://{ip}:{port}',
        }
        print(proxies)
        # 存入队列
        q.put(proxies)
    main()

print(ip_list)
print("----结束----")

# 章节5-2 图片爬虫
把千图网（https://www.58pic.com/ ）某个频道的所有图片爬下来，高清原版的。(网址变了)

In [ ]:
import urllib.request
import re

headers = ("User-Agent", "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36")
opener = urllib.request.build_opener()
opener.addheaders=[headers]
urllib.request.install_opener(opener)

for i in range(1,2):
#     pageurl = "https://www.58pic.com/haibaomoban/0/id-{}.html".format(str(i*25))
    pageurl = "https://www.58pic.com/tupian/so-0-0-default-0-0-SO-0_10_0_0_0_0_0-0-{}.html".format(str(i))
    data = urllib.request.urlopen(pageurl).read().decode("utf-8", "ignore")
    pat = 'data-original="//(preview.qiantucdn.com/.*?new_nowater)"'
    imglist = re.compile(pat).findall(data)
    for j in range(0, len(imglist)):
        try:
            thisimg = imglist[j]
#             thisimgurl = thisimg + "_1024.jpg"
            thisimgurl = "http://" + thisimg
            file = "img/" + str(i) + str(j) + ".jpg"
            urllib.request.urlretrieve(thisimgurl, filename = file)
            print("----爬取第{}页第{}个图片成功----".format(str(i+1), str(j+1)))
        except urllib.error.URLError as e:
            if hasattr(e, "code"):
                print(e.code)
            if hasattr(e, "reason"):
                print(e.reason)
        except Exception as e:
            print(e)
            

爬取豆瓣电影Top250（https://movie.douban.com/top250 ）。

In [ ]:
import urllib.request
import re

# keyname="短裤"
# key = urllib.request.quote(keyname)
headers = ("User-Agent", "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36")
opener = urllib.request.build_opener()
opener.addheaders=[headers]
urllib.request.install_opener(opener)
print("----开始爬取----")
for i in range(0, 10):
    url = "https://movie.douban.com/top250?start={0}&filter=".format(str(i*25))
    data = urllib.request.urlopen(url).read().decode("utf-8", "ignore")
    pat = 'src="(https://img.*?)" class=""'
    imageurl_list = re.compile(pat).findall(data)
    for j in range(len(imageurl_list)):
        thisimg = imageurl_list[j]
        file = "img/" + str(i) + str(j) + ".jpg"
        urllib.request.urlretrieve(thisimg, filename = file)
    print("----爬取第{}页成功----".format(str(i+1)))

print("爬取网页列表为：" + str(imageurl))

# 章节6-1 抓包分析
爬取腾讯视频（青云志）前20条评论（深度解读）。

In [ ]:
import urllib.request
import re

headers = ("User-Agent", "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36")
opener = urllib.request.build_opener()
opener.addheaders=[headers]
urllib.request.install_opener(opener)
comid, total = "", 0
url = 'https://video.coral.qq.com/filmreviewr/c/upcomment/g7jpq0qd0k8xdca?callback=_filmreviewrcupcommentg7jpq0qd0k8xdca&reqnum=3&source=132&commentid=' + comid + "&_=1605598626157"
while total <= 20:
    try:
        data = urllib.request.urlopen(url, timeout = 10).read().decode()
    except urllib.error.URLError as e:
        if hasattr(e, "code"):
            print(e.code)
        if hasattr(e, "reason"):
            print(e.reason)
    except Exception as e:
        print(e)
    patnext = '"last":"(.*?)"'
    nextid = re.compile(patnext).findall(data)[0]
    pattitle = '"title":"(.*?)",'
    comtitle = re.compile(pattitle).findall(data)
    patcom = '"content":"(.*?)",'
    comdata = re.compile(patcom).findall(data)
    for j in range(len(comdata)):
        total += 1
        if total > 20:
            break
        print("----第{0}条评论----".format(str(total)))
        title = eval('u"' + comtitle[j] + '"') 
        print("标题：" + title) 
        content = eval('u"' + comdata[j] + '"')
        print("内容：" + content)
    url = 'https://video.coral.qq.com/filmreviewr/c/upcomment/g7jpq0qd0k8xdca?callback=_filmreviewrcupcommentg7jpq0qd0k8xdca&reqnum=3&source=132&commentid=' + nextid + "&_=1605598626157"
print("----结束爬虫----")

# 章节6-3 微信爬虫实战
使用fiddler代理服务器爬取微信网页（https://weixin.sogou.com/ ）信息。

In [ ]:
import re
import time
import urllib.request
import urllib.error

# 自定义函数，功能为使用代理服务器爬一个网址
def use_proxy(proxy_addr, url):
    try:
        req = urllib.request.Request(url)
        req.add_header('User-Agent', 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36')
        proxy = urllib.request.ProxyHandler({'http': proxy_addr})
        opener = urllib.request.build_opener(proxy, urllib.request.HTTPHandler)
        urllib.request.install_opener(opener)
        data = urllib.request.urlopen(req).read()
        return data
    except urllib.error.URLError as e:
        if hasattr(e, 'code'):
            print(e.code)
        if hasattr(e, 'reason'):
            print(e.reason)
        time.sleep(10)
    except Exception as e:
        print("Exception:" + str(e))
        time.sleep(1)

key = "python"
proxy_addr = '127.0.0.1:8888'
for i in range(0, 10):
    key = urllib.request.quote(key)
    thispageurl = "https://weixin.sogou.com/weixin?type=2&query={}&page={}".format(key, str(i))
    thispagedata = use_proxy(proxy, thispageurl)
    print(len(str(thispagedata)))
    pat1 = '<a href="(.*?)"'
    rs1 = re.compile(pat1, re.S).findall(str(thispagedata))
    if(len(rs1) == 0):
        print("此次{}页没爬取成功。".format(str(i)))
        continue
    for j in range(0, len(rs1)):
        thisurl = rs1[j]
        thisurl = thisurl.replace("amp;", "")
        file = "/result/第{}页第{}篇文章.html".format(str(i), str(j))
        thisdata = use_proxy(proxy, thisurl)
        try:
            fh = open(file, "wb")
            fh.write(thisdata)
            fh.close()
            print("第{}页第{}篇文章成功".format(str(i), str(j)))
        except Exception as e:
            print(e)
            print("第{}页第{}篇文章失败".format(str(i), str(j)))

PS.微信改版，旧的代码已经不可用，新的请参考。https://www.cnblogs.com/hyonline/p/11812977.html
涉及反爬技术：Cookie构造和js加密

In [ ]:
import requests
from lxml import etree
import re
import random
import json
from urllib import parse


def get_cookie(response1, uigs_para, UserAgent):
    SetCookie = response1.headers['Set-Cookie']
    cookie_params = {
        "ABTEST": re.findall('ABTEST=(.*?);', SetCookie, re.S)[0],
        "SNUID": re.findall('SNUID=(.*?);', SetCookie, re.S)[0],
        "IPLOC": re.findall('IPLOC=(.*?);', SetCookie, re.S)[0],
        "SUID": re.findall('SUID=(.*?);', SetCookie, re.S)[0]
    }
    
    url = "https://www.sogou.com/sug/css/m3.min.v.7.css"
    headers = {
        "Accept": "text/css,*/*;q=0.1",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2",
        "Connection": "keep-alive",
        "Cookie": "SNUID={}; IPLOC={}".format(cookie_params['SNUID'], cookie_params['IPLOC']),
        "Host": "www.sogou.com",
        "Referer": "https://weixin.sogou.com/",
        "User-Agent": UserAgent
    }
    response2 = requests.get(url, headers=headers)
    SetCookie = response2.headers['Set-Cookie']
    cookie_params['SUID'] = re.findall('SUID=(.*?);', SetCookie, re.S)[0]
    
    url = "https://weixin.sogou.com/websearch/wexinurlenc_sogou_profile.jsp"
    headers = {
        "Accept": "*/*",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2",
        "Connection": "keep-alive",
        "Cookie": "ABTEST={}; SNUID={}; IPLOC={}; SUID={}".format(cookie_params['ABTEST'], cookie_params['SNUID'], cookie_params['IPLOC'],
                                                                  cookie_params['SUID']),
        "Host": "weixin.sogou.com",
        "Referer": response1.url,
        "User-Agent": UserAgent
    }
    response3 = requests.get(url, headers=headers)
    SetCookie = response3.headers['Set-Cookie']
    cookie_params['JSESSIONID'] = re.findall('JSESSIONID=(.*?);', SetCookie, re.S)[0]
    
    url = "https://pb.sogou.com/pv.gif"
    headers = {
        "Accept": "image/webp,*/*",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2",
        "Connection": "keep-alive",
        "Cookie": "SNUID={}; IPLOC={}; SUID={}".format(cookie_params['SNUID'], cookie_params['IPLOC'], cookie_params['SUID']),
        "Host": "pb.sogou.com",
        "Referer": "https://weixin.sogou.com/",
        "User-Agent": UserAgent
    }
    response4 = requests.get(url, headers=headers, params=uigs_para)
    SetCookie = response4.headers['Set-Cookie']
    cookie_params['SUV'] = re.findall('SUV=(.*?);', SetCookie, re.S)[0]
    
    return cookie_params


def get_k_h(url):
    b = int(random.random() * 100) + 1
    a = url.find("url=")
    url = url + "&k=" + str(b) + "&h=" + url[a + 4 + 21 + b: a + 4 + 21 + b + 1]
    return url


def get_uigs_para(response):
    uigs_para = re.findall('var uigs_para = (.*?);', response.text, re.S)[0]
    if 'passportUserId ? "1" : "0"' in uigs_para:
        uigs_para = uigs_para.replace('passportUserId ? "1" : "0"', '0')
    uigs_para = json.loads(uigs_para)
    exp_id = re.findall('uigs_para.exp_id = "(.*?)";', response.text, re.S)[0]
    uigs_para['right'] = 'right0_0'
    uigs_para['exp_id'] = exp_id[:-1]
    return uigs_para


def main_v4(list_url, UserAgent):
    headers1 = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8",
        "Connection": "keep-alive",
        "Host": "weixin.sogou.com",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": UserAgent,
    }
    response1 = requests.get(list_url, headers=headers1)
    html = etree.HTML(response1.text)
    urls = ['https://weixin.sogou.com' + i for i in html.xpath('//div[@class="img-box"]/a/@href')]
    
    uigs_para = get_uigs_para(response1)
    params = get_cookie(response1, uigs_para, UserAgent)
    approve_url = 'https://weixin.sogou.com/approve?uuid={}'.format(uigs_para['uuid'])
    headers2 = {
        "Accept": "*/*",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2",
        "Connection": "keep-alive",
        "Cookie": "ABTEST={}; IPLOC={}; SUID={}; SUV={}; SNUID={}; JSESSIONID={};".format(params['ABTEST'], params['IPLOC'],
                                                                                          params['SUID'], params['SUV'], params['SNUID'],
                                                                                          params['JSESSIONID']),
        "Host": "weixin.sogou.com",
        "Referer": response1.url,
        "User-Agent": UserAgent,
        "X-Requested-With": "XMLHttpRequest"
    }
    for url in urls:
        response2 = requests.get(approve_url, headers=headers2)
        url = get_k_h(url)
        headers3 = {
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
            "Accept-Encoding": "gzip, deflate, br",
            "Accept-Language": "zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2",
            "Connection": "keep-alive",
            "Cookie": "ABTEST={}; SNUID={}; IPLOC={}; SUID={}; JSESSIONID={}; SUV={}".format(params['ABTEST'], params['SNUID'],
                                                                                             params['IPLOC'], params['SUID'],
                                                                                             params['JSESSIONID'],
                                                                                             params['SUV']),
            "Host": "weixin.sogou.com",
            "Referer": list_url,
            "Upgrade-Insecure-Requests": "1",
            "User-Agent": UserAgent
        }
        response3 = requests.get(url, headers=headers3)
        
        fragments = re.findall("url \+= '(.*?)'", response3.text, re.S)
        itemurl = ''
        for i in fragments:
            itemurl += i
        
        # 文章url拿正文
        headers4 = {
            "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3",
            "accept-encoding": "gzip, deflate, br",
            "accept-language": "zh-CN,zh;q=0.9,en;q=0.8",
            "cache-control": "max-age=0",
            "user-agent": UserAgent
        }
        response4 = requests.get(itemurl, headers=headers4)
        html = etree.HTML(response4.text)
        print(response4.status_code)
        print(html.xpath('//meta[@property="og:title"]/@content')[0])


if __name__ == "__main__":
    key = "咸蛋超人"
    url = 'https://weixin.sogou.com/weixin?type=2&s_from=input&query={}&_sug_=n&_sug_type_=&page=1'.format(parse.quote(key))
    UserAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:70.0) Gecko/20100101 Firefox/70.0"
    main_v4(url, UserAgent)

# 章节6-4 多线程爬虫

In [ ]:
import urllib.request
import re
import urllib.error

headers = ("User-Agent", "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36")
opener = urllib.request.build_opener()
opener.addheaders=[headers]
urllib.request.install_opener(opener)
for i in range(1,3):
    url = "https://www.qiushibaike.com/8hr/page/" + str(i)
    pagedata = urllib.request.urlopen(url).read().decode("utf-8", "ignore")
    pat = '<div class="content">.*?<span>(.*?)</span>.*?</div>'
    datalist = re.compile(pat, re.S).findall(pagedata)
    for j in range(0, len(datalist)):
        print("第" + str(i) + "页第" + str(j) + "个段子的内容是：")
        print(datalist[j])
    
    

In [ ]:
import threading

class A(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
    def run(self):
        for i in range(10):
            print("我是线程A")
class B(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
    def run(self):
        for i in range(10):
            print("我是线程B")
t1 = A()
t1.start()
t2 = B()
t2.start()
            

# 章节6-5 Scrapy框架的安装



# 章节8-1 天善智能课程自动爬虫

爬虫难点：1、反爬处理；2、抓包；3、分布式与多线程。
验证码处理：1、图像识别；2、外部接口；3、半自动方式。

In [ ]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
link = 'http://www.santostang.com/'   # 定义link为目标网页地址
# 定义请求头的浏览器代理，进行伪装。
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
# r是requests的Response回复对象
r = requests.get(link, headers = headers)
# print(r.text)  # r.text是获取的网页内容代码
soup = BeautifulSoup(r.text, "html.parser")  # 使用BeautifulSoup解析
title = soup.find("h1", class_="post-title").a.text.strip()
print(title)
html = etree.HTML(r.text)  # 使用xpath解析
title = html.xpath('//*[@id="main"]/div/div[1]/article[1]/header/h1/a/text()')[0]
print(title)
# 存储数据到txt
with open('test.txt', 'a+') as f:
    f.write(title)

In [ ]:
import requests

r = requests.get('http://www.santostang.com/')
print('文本编码：' , r.encoding)
print('响应状态码：' , r.status_code)
print('JSON解码器：' , r.json)
print('字节方式的响应体：' , r.content)
print('字符串方式的响应体：' , r.text)

In [ ]:
import requests

key_dict = {'key1': 'value1', 'key2': 'value2'}
r = requests.get('http://httpbin.org/get', params=key_dict)
print("URL已经正确编码：", r.url)
print("字符串方式的响应体：\n", r.text)

In [ ]:
import requests
headers = {'Host': 'www.santostang.com', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
r = requests.get('http://www.santostang.com', headers=headers)
print('响应状态码：', r.status_code)

In [ ]:
import requests
key_dict = {'key1': 'value1', 'key2': 'value2'}
r = requests.post('http://httpbin.org/post', data=key_dict)
print(r.text)

In [ ]:
import requests
link = "http://www.santostang.com/"
r = requests.get(link, timeout = 0.01)

In [ ]:
import requests
from lxml import etree

def get_movies():
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
    movie_list = []
    for i in range(0, 10):
        link = 'https://movie.douban.com/top250?start=' + str(i*25)
        r = requests.get(link, headers=headers, timeout=20)
        print(str(i+1), '页响应状态码：', r.status_code)
        html = etree.HTML(r.text)
        title_list = html.xpath('//div[@class="hd"]/a/span[1]/text()')
        for each in title_list:
            movie_list.append(each)
    return movie_list

movies = get_movies()
print(movies)

In [ ]:
import requests
import json

def single_page_comment(link):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
    r = requests.get(link, headers=headers)
    json_string = r.text
    json_string = json_string[json_string.find('{'):-2]
    json_data = json.loads(json_string)
    comment_list = json_data['results']['parents']
    for each in comment_list:
        message = each['content']
        print(message)
    
for page in range(1, 4):
    link1 = "https://api-zero.livere.com/v1/comments/list?callback=jQuery112400642670146062918_1610003898881&limit=10&offset="
    link2 = "&repSeq=4272904&requestPath=%2Fv1%2Fcomments%2Flist&consumerSeq=1020&livereSeq=28583&smartloginSeq=5154&code="
    link = link1 + str(page) + link2
    print("第"+str(page)+"页: ", link)
    single_page_comment(link)
    print("\n")

In [ ]:
from selenium import webdriver
import time

driver = webdriver.Firefox(executable_path = r'D:\Program Files\geckodriver\geckodriver.exe')
driver.implicitly_wait(20) # 隐性等待，最长等20秒
driver.get("http://www.santostang.com/2018/07/04/hello-world/")
# time.sleep(5)
# for i in range(0,3):
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 下滑到页面底部
#     # 把所有评论加载出来，首先转换iframe，再找到查看更多，点击
#     driver.switch_to.frame(driver.find_element_by_css_selector("iframe[title='livere-comment']"))
#     load_more = driver.find_element_by_css_selector('button.more-btn')
#     load_more.click()
#     # 把iframe又转回去
#     driver.switch_to.default_content()
#     time.sleep(2)

driver.switch_to.frame(driver.find_element_by_css_selector("iframe[title='livere-comment']"))
comments = driver.find_elements_by_css_selector('div.reply-content')
# print(comment)
for eachcomment in comments:
    content = eachcomment.find_element_by_tag_name('p')
    print(content.text)

In [ ]:
user = dirver.find_element_by_name("username")
user.clear # 清除元素的内容、
user.send_keys("123456") # 模拟按键输入
pwd = driver.find_element_by_name("password") # 找到密码输入框
pwd.clear
pwd.send_keys("123456")
dirver.find_element_by_id("loginBtn").click() # 单击登录

In [ ]:
//*[@id="listing-46887483"]/div[2]/div[2]/div/div/div[1]/div/span/span/span/span/span[2]
https://www.airbnb.cn/s/Shenzhen--China/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&screen_size=large&hide_dates_and_guests_filters=false&place_id=ChIJkVLh0Aj0AzQRyYCStw1V7v0&map_toggle=false
https://www.airbnb.cn/s/Shenzhen--China/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&screen_size=large&hide_dates_and_guests_filters=false&map_toggle=false&place_id=ChIJkVLh0Aj0AzQRyYCStw1V7v0&s_tag=LBUutDen&last_search_session_id=b293d63f-e3fe-4543-8bea-509eff66f7b1&items_offset=20&section_offset=6
https://www.airbnb.cn/s/Shenzhen--China/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&screen_size=medium&hide_dates_and_guests_filters=false&map_toggle=false&s_tag=LBUutDen&place_id=ChIJkVLh0Aj0AzQRyYCStw1V7v0&last_search_session_id=ddabba5b-96eb-4aa1-b863-255c8759097f&items_offset=40&section_offset=6

In [ ]:
from selenium import webdriver
import time

driver = webdriver.Firefox(executable_path = r'D:\Program Files\geckodriver\geckodriver.exe')
#把上述地址改成你电脑中geckodriver.exe程序的地址
#在虚拟浏览器中打开 Airbnb 页面
driver.get("https://zh.airbnb.com/s/Shenzhen--China/homes")

#找到页面中所有的出租房
rent_list = driver.find_elements_by_css_selector('div._gig1e7')

#对于每一个出租房
for eachhouse in rent_list:
    #找到评论数量
    try:
        comment = eachhouse.find_element_by_css_selector('span._69pvqtq')
        comment = comment.text
    except:
        comment = 0
    
    #找到价格
#     price = eachhouse.find_element_by_css_selector('span._1d8yint7')
#     price = price.text.replace("每晚", "").replace("价格", "").replace("\n", "")
    
    #找到名称
    name = eachhouse.find_element_by_css_selector('div._qrfr9x5')
    name = name.text
    
    #找到房屋类型，大小
    details = eachhouse.find_element_by_css_selector('span._faldii7')
    details = details.text
    house_type = details.split(" · ")[0]
    bed_number = details.split(" · ")[1]
    print (comment, name, house_type, bed_number)

In [ ]:
from selenium import webdriver
import time

driver = webdriver.Firefox(executable_path = r'D:\Program Files\geckodriver\geckodriver.exe')
#把上述地址改成你电脑中geckodriver.exe程序的地址
for i in range(0,2):
    link = "https://zh.airbnb.com/s/Shenzhen--China/homes?items_offset=" + str(i * 20)
    driver.get(link)
    rent_list = driver.find_elements_by_css_selector('div._gig1e7')

    for eachhouse in rent_list:
        try:
            comment = eachhouse.find_element_by_css_selector('span._1clmxfj').text
        except:
            comment = '缺失'
        try:
            price = eachhouse.find_element_by_css_selector('div._1ixtnfc')
            price = price.text.replace("每晚", "").replace("价格", "").replace("\n", "")
        except:
            price = '缺失'
        try:
            name = eachhouse.find_element_by_css_selector('div._qrfr9x5')
            name = name.text
        except:
            name = '缺失'
        try:
            details = eachhouse.find_element_by_css_selector('span._faldii7')
            details = details.text
            house_type = details.split(" · ")[0]
            bed_number = details.split(" · ")[1]
        except:
            house_type, bed_number = '缺失', '缺失'
        print (comment, price, name, house_type, bed_number)
    time.sleep(5)

In [ ]:
import re
pattern = '[0-9]+'
line = 'The first number is 12345, 12345 is the second.'
print(re.match(pattern, line))
print(re.search(pattern, line))
print(re.findall(pattern, line))

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.98 Safari/537.36'}
for i in range(1,11):
    link = 'https://beijing.anjuke.com/sale/p' + str(i)
    r = requests.get(link, headers = headers)
    print ('现在爬取的是第', i, '页')

    soup = BeautifulSoup(r.text, 'lxml')
    house_list = soup.find_all('li', class_="list-item")

    for house in house_list:
        try:
            name = house.find('div', class_ ='house-title').a.text.strip()
            price = house.find('span', class_='price-det').text.strip()
            price_area = house.find('span', class_='unit-price').text.strip()

            no_room = house.find('div', class_='details-item').span.text
            area = house.find('div', class_='details-item').contents[3].text
            floor = house.find('div', class_='details-item').contents[5].text
            year = house.find('div', class_='details-item').contents[7].text
            broker = house.find('span', class_='broker-name broker-text').text
            broker = broker[1:]
            address = house.find('span', class_='comm-address').text.strip()
            address = address.replace('\xa0\xa0\n                    ','  ')
            tag_list = house.find_all('span', class_='item-tags')
            tags = [i.text for i in tag_list] 
            print (name, price, price_area, no_room, area, floor, year, broker, address, tags)
        except Exception as e:
            print(e)
    time.sleep(5)

In [4]:
import pymysql
# 打开数据库连接
conn = pymysql.connect(host="127.0.0.1", user="root", password="root123456.", db="dangdang")
# SQL操作
sql = "INSERT INTO books(title, link, cost, bookshop, comment) VALUES(%s, %s, %s, %s, %s)"
param = ("test2", "test2", "test", "test", "test")
try:
    # 使用cursor()方法获取操作游标
    cursor = conn.cursor()
    # 执行sql语句
    cursor.execute(sql, param)
    # 提交到数据库执行
    conn.commit()
    print("插入成功！")
except Exception as e:
    print(e)
    # 如果发生错误则回滚
    conn.rollback()
finally:
    # 关闭数据库连接
    cursor.close()
    conn.close()

插入成功！


In [1]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.blog_database
collection = db.blog

In [3]:
import requests
import datetime
from bs4 import BeautifulSoup
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.blog_database
collection = db.blog
link = "http://www.santostang.com/"
headers = {'User-Agent' : 'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6'} 
r = requests.get(link, headers= headers)
soup = BeautifulSoup(r.text, "lxml")
title_list = soup.find_all("h1", class_="post-title")
for eachone in title_list:
    url = eachone.a['href']
    title = eachone.a.text.strip()
    post = {"url": url, "title": title, "data": datetime.datetime.utcnow()}
    collection.insert_one(post)

In [5]:
import requests
from bs4 import BeautifulSoup
import datetime
from pymongo import MongoClient

# 获取页面
def get_page(link):
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6'} 
    r = requests.get(link, headers = headers)
    html = r.content  #使用r.content解封装
    html = html.decode('utf-8')  #由UTF-8解码为unicode
    soup = BeautifulSoup(html, 'lxml')
    return soup

# 解析网页
def get_data(post_list):
    data_list =[]
    for post in post_list:
        title = post.find('div',class_='titlelink box').text.strip()
        post_link = post.find('div',class_='titlelink box').a['href']
        post_link = "https://bbs.hupu.com" + post_link 

        author = post.find('div',class_='author box').a.text.strip()
        author_page = post.find('div',class_='author box').a['href']
        start_date = post.find('div',class_='author box').contents[5].text.strip()

        reply_view = post.find('span',class_='ansour box').text.strip()
        reply = reply_view.split('/')[0].strip()
        view = reply_view.split('/')[1].strip()

        reply_time = post.find('div',class_='endreply box').a.text.strip()
        last_reply = post.find('div',class_='endreply box').span.text.strip()
        if ':' in reply_time: #时间是11:27
            date_time = str(datetime.date.today()) + ' ' + reply_time
            date_time = datetime.datetime.strptime(date_time, '%Y-%m-%d %H:%M')
        elif reply_time.find("-") == 4: #时间是2017-02-27
            date_time = datetime.datetime.strptime(reply_time, '%Y-%m-%d').date()
        else: #时间是11-27
            date_time = datetime.datetime.strptime('2018-' + reply_time, '%Y-%m-%d').date()
        data_list.append([title, post_link, author, author_page, start_date, reply, view, last_reply, date_time])
    return data_list

class MongoAPI(object):
    def __init__(self, db_ip, db_port, db_name, table_name):
        self.db_ip = db_ip
        self.db_port = db_port
        self.db_name = db_name
        self.table_name = table_name
        self.conn = MongoClient(host=self.db_ip, port=self.db_port)
        self.db = self.conn[self.db_name]
        self.table = self.db[self.table_name]
    def get_one(self, query):
        return self.table.find_one(query, projection={"_id": False})
    def get_all(self, query):
        return self.table.find(query)
    def add(self, kv_dict):
        return self.table.insert_one(kv_dict)
    def delete(self, query):
        return self.table.delete_many(query)
    def check_exist(self, query):
        ret = self.table.find_one(query)
        return ret != None
    # 如果没有会新建
    def update(self, query, kv_dict):
            self.table.update_one(query,{
              '$set': kv_dict
            }, upsert=True)

link = "https://bbs.hupu.com/bxj"
soup = get_page(link)
post_all= soup.find('ul', class_="for-list")
post_list = post_all.find_all('li')
data_list = get_data(post_list)
for each in data_list:
    print (each)
    
hupu_post = MongoAPI("localhost",  27017,  "hupu", "post")
for each in data_list:
    hupu_post.update({"post_link": each[1]},{"title": each[0], 
                    "post_link": each[1],
                   "author": each[2],
                   "author_page": each[3],
                   "start_date": str(each[4]),
                   "reply": each[5],
                   "view": each[6],
                   "last_reply": each[7],
                   "last_reply_time": str(each[8])})

['[置顶]\n【虎扑-天选之子】冬日送温暖，iphone12mini免费送\n\xa0\n\n\xa0\n\n[\xa0\n2\n3...15\n\n\xa0]', 'https://bbs.hupu.com/40346769.html', '天选之子官方', 'https://my.hupu.com/242532589161998', '2021-01-08', '288', '2186553', '满眼星辰皆是陳', datetime.datetime(2021, 1, 12, 16, 39)]
['学校这位卧龙什么水平？\n\n\xa0\n\n[\xa0\n2\n3...8\n\n\xa0]', 'https://bbs.hupu.com/40424541.html', '说不过就别接着杠啦', 'https://my.hupu.com/130313337891224', '2021-01-12', '143', '155538', '虎扑JR1911883317', datetime.datetime(2021, 1, 12, 16, 45)]
['《传奇》这游戏已经有20多年了吧，咋还有人玩呢，好在哪。。。\n\n\xa0\n\n[\xa0\n2\n3...9\n\n\xa0]', 'https://bbs.hupu.com/40402904.html', '一炮干哈炕', 'https://my.hupu.com/208290034176260', '2021-01-11', '169', '102834', '山羊不如绵羊', datetime.datetime(2021, 1, 12, 16, 45)]
['女朋友被陌生人在微博侮辱，只能请家人们出主意了！ \n\n\xa0\n\n[\xa0\n2\n3...7\n\n\xa0]', 'https://bbs.hupu.com/40422577.html', '麦子灬马刺', 'https://my.hupu.com/87691030988142', '2021-01-12', '129', '43049', '20200911秃了也变憨了', datetime.datetime(2021, 1, 12, 16, 45)]
['什么时候才能真正的长大？该怎么教育？\n\n\xa0\n\n[\xa0\